# Times Higher Education World Reputation Rankings 2023

Top 100 University scores + descriptions

In [ ]:
! pip install selenium webdriver_manager==4.0.2

In [28]:
import pandas as pd
import os
import logging


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

In [54]:
logging.getLogger('WDM').setLevel(logging.NOTSET)
os.environ['WDM_LOG'] = 'False'

# Driver set-up
service = Service(ChromeDriverManager().install().replace("THIRD_PARTY_NOTICES.", ''))
driver = webdriver.Chrome(service=service)

In [58]:
DATA = []


def get_description(url):
  driver.get(url)
  try:
    driver.find_element(By.XPATH, './/button[text()="Allow all"]').click()
  except:
    pass
  driver.find_element(By.XPATH, './/p[text()="more"]').click()
  text_block = driver.find_element(By.XPATH, './/*[@data-testid="rich-text"]')
  paragraphs = text_block.find_elements(By.CSS_SELECTOR, 'p')
  description = ' '.join([p.text for p in paragraphs])
  return description


def get_ranking(page_url):
  driver.get(page_url)
  try:
    driver.find_element(By.XPATH, './/button[text()="Allow all"]').click()
  except:
    pass
  table = driver.find_element(By.XPATH, './/*[@data-module="rankings_table"]').find_element(By.CSS_SELECTOR, 'tbody')
  rows = table.find_elements(By.XPATH, './/*[@role="row"]')
  for row in rows:
    rank = row.find_element(By.XPATH, './/*[@class="rank sorting_1 sorting_2"]').text
    location = row.find_element(By.XPATH, './/*[@class="location"]').text
    overall = row.find_element(By.XPATH, './/*[@class=" scores overall-score"]').text
    research = row.find_element(By.XPATH, './/*[@class=" scores research-score"]').text
    teaching = row.find_element(By.XPATH, './/*[@class=" scores teaching-score"]').text
    title = row.find_element(By.XPATH, './/*[@class="ranking-institution-title"]')
    name = title.text
    description_url = title.get_attribute('href')
    
    data_dict = {
            'Rank' : rank,
            'Name' : name,
            'Location' : location,
            'Description': description_url,
            'Overall Reputation Score' : overall,
            'Research Reputation Score' : research,
            'Teaching Reputation Score' : teaching
        }
    DATA.append(data_dict)
    print(rank, name)

In [ ]:
get_ranking("https://www.timeshighereducation.com/world-university-rankings/2023/reputation-ranking#!/length/100/sort_by/rank/sort_order/asc")

In [48]:
DATA_DF = pd.DataFrame(DATA)

,Rank,Name,Location,Description,Overall Reputation Score,Research Reputation Score,Teaching Reputation Score
0,1,Harvard University,United States,https://www.timeshighereducation.com/world-uni...,100.0,100.0,100.0
1,2,Massachusetts Institute of Technology,United States,https://www.timeshighereducation.com/world-uni...,89.6,93.9,77.1
2,3,Stanford University,United States,https://www.timeshighereducation.com/world-uni...,81.1,84.0,72.4
3,4,University of Oxford,United Kingdom,https://www.timeshighereducation.com/world-uni...,80.5,81.1,78.7
4,5,University of Cambridge,United Kingdom,https://www.timeshighereducation.com/world-uni...,77.6,78.5,75.0
...,...,...,...,...,...,...,...
95,91-100,Michigan State University,United States,https://www.timeshighereducation.com/world-uni...,-,-,-
96,91-100,Ohio State University (Main campus),United States,https://www.timeshighereducation.com/world-uni...,-,-,-
97,91-100,Politecnico di Milano,Italy,https://www.timeshighereducation.com/world-uni...,-,-,-
98,91-100,The University of Queensland,Australia,https://www.timeshighereducation.com/world-uni...,-,-,-


In [ ]:
TEXTS = []
for i, link in enumerate(DATA_DF["Description"][16:]):
    try:
        text = get_description(link)
        TEXTS[i + 16] = text
    except Exception as e:
        TEXTS[i + 16] = '-'

In [61]:
DATA_DF["Description"] = TEXTS
DATA_DF["Description"] = DATA_DF["Description"].apply(str.strip)

In [63]:
DATA_DF.to_csv("university_ranking.csv")